In [2]:
import aiohttp

async def getToken(username, password):
    keyurl = "http://localhost:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            # print(resp.status)
            keyJson = await resp.json()
            # print(keyJson)

        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            # print(resp.status)
            tokenJson = await resp.json()
            # print(tokenJson)
    return tokenJson.get("token", None)

           

def query(q, token):
    async def post(variables):
        gqlurl = "http://localhost:33001/api/gql"
        payload = {"query": q, "variables": variables}
        # headers = {"Authorization": f"Bearer {token}"}
        cookies = {'authorization': token}

        async with aiohttp.ClientSession() as session:
            # print(headers, cookies)
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                # print(resp.status)
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text

                else:
                    response = await resp.json()
                    return response

    return post

In [12]:
from itertools import product
from functools import reduce

def enumerateAttrs(attrs):
    for key, value in attrs.items():
        names = value.split(".")
        name = names[0]
        yield key, name

 

def flattenList(inList, outItem, attrs):
    for item in inList:
        assert isinstance(item, dict), f"in list only dicts are expected"
        for row in flatten(item, outItem, attrs):
            # print("flatList", row)
            yield row

           

def flattenDict(inDict, outItem, attrs):
    result = {**outItem}
    # print("flatDict.result", result)
    complexAttrs = []
    for key, value in enumerateAttrs(attrs):
        attributeValue = inDict.get(value, None)
        if isinstance(attributeValue, list):
            complexAttrs.append((key, value))
        elif isinstance(attributeValue, dict):
            complexAttrs.append((key, value))
        else:
            result[key] = attributeValue
    lists = []

    for key, value in complexAttrs:
        attributeValue = inDict.get(value, None)
        prefix = f"{value}."
        prefixlen = len(prefix)
        subAttrs = {key: value[prefixlen:] for key, value in attrs.items() if value.startswith(prefix)}
        items = list(flatten(attributeValue, result, subAttrs))
        lists.append(items)
                   

    if len(lists) == 0:
        yield result

    else:
        for element in product(*lists):
            reduced = reduce(lambda a, b: {**a, **b}, element, {})
            yield reduced

       

def flatten(inData, outItem, attrs):
    if isinstance(inData, dict):
        for item in flattenDict(inData, outItem, attrs):
            yield item

    elif isinstance(inData, list):
        for item in flattenList(inData, outItem, attrs):
            yield item
    else:
        assert False, f"Unexpected type on inData {inData}"

In [4]:
username = "john.newbie@world.com"

password = "john.newbie@world.com"

 

queryStr = """

{

  result: userPage(

    where: {memberships: {group: {name: {_ilike: "%uni%"}}}}

  ) {

    id

    email

    name

    surname

   

    presences {

      event {

        id

        name

        startdate

        enddate

        eventType {

          id

          name

        }

      }

      presenceType {

        id

        name

      }

    }
  }
}
"""

 

mappers = {

    "id": "id",
    "name": "name",
    "surname": "surname",
    "email": "email",
    "eventid": "presences.event.id",
    "eventname": "presences.event.name",
    "startdate": "presences.event.startdate",
    "enddate": "presences.event.enddate",
    "eventTypeid": "presences.event.eventType.id",
    "eventTypename": "presences.event.eventType.name",
    "presenceTypeid": "presences.presenceType.id",
    "presenceTypename": "presences.presenceType.name"
}


async def fullPipe():
    global pandasData
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})

    data = response.get("data", None)
    result = data.get("result", None)
    # pandasData = toTable(flatData)
    flatData = flatten(result, {}, mappers)
    # print(list(flatData))
    for item in flatData:
        print(item)
  
    return flatData

 


# print(list(flatData))

In [6]:
result =await fullPipe()
print(result)

{'id': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'name': 'Jana', 'surname': 'Newbie', 'email': 'jana.newbie@world.com', 'eventid': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventname': 'Zkouška', 'startdate': '2023-04-19T08:00:00', 'enddate': '2023-04-19T09:00:00', 'eventTypeid': 'b87d3ff0-8fd4-11ed-a6d4-0242ac110002', 'eventTypename': 'Ostatní', 'presenceTypeid': '466398c6-a79c-11ed-b76e-0242ac110002', 'presenceTypename': 'Přítomen'}
{'id': '89d1f34a-ae0f-11ed-9bd8-0242ac110002', 'name': 'Jolana', 'surname': 'Newbie', 'email': 'jolana.newbie@world.com', 'eventid': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventname': 'Zkouška', 'startdate': '2023-04-19T08:00:00', 'enddate': '2023-04-19T09:00:00', 'eventTypeid': 'b87d3ff0-8fd4-11ed-a6d4-0242ac110002', 'eventTypename': 'Ostatní', 'presenceTypeid': '466398c6-a79c-11ed-b76e-0242ac110002', 'presenceTypename': 'Přítomen'}
{'id': '89d1f3cc-ae0f-11ed-9bd8-0242ac110002', 'name': 'Jitka', 'surname': 'Newbie', 'email': 'jitka.newbie@world.com', 'eve

In [83]:
username = "john.newbie@world.com"

password = "john.newbie@world.com"

 
#ad where in publicationpage
queryStr = """
query{
  result: publicationPage{
    id
    name
    publicationtype {
      id
      name

    }
    authors{
      id
      order
      lastchange
      share
      valid
      user{
        id
        name
        surname
        email
      }
    }
    place
    publishedDate
    valid
    reference
    subjects {
      id
      name
    }
    
  }
  
}
"""

mappers = {
    "id": "id",
    "namePublication": "name",
    "typePublication": "publicationtype.name",
    "place":"place",
    "publishedDate":"publishedDate",
    "valid":"valid",
    "reference":"reference",
    # "authorName":"authors.user.name",
    # "authorOrder":"authors.user.order",
    # "authorContribute":"authors.user.share"

}

 

In [1]:
async def fullPipe2():
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})

    data = response.get("data", None)
    result = data.get("result", None)
    # pandasData = toTable(flatData)
    flatData = flatten(result, {}, mappers)
    # print(list(flatData))
    for item in flatData:
        print(item)
    

    return flatData

In [11]:
result2 =await fullPipe2()
print(result2)

{'id': 'cb3c3978-e716-46ac-9a3b-bb8f9d806a46', 'namePublication': 'IT Technologie', 'place': 'Prague', 'publishedDate': '2023-04-24T00:00:00', 'valid': True, 'reference': 'https://google.com', 'typePublication': 'Článek J'}
{'id': 'e8b1962f-5494-40cf-9bd6-b7d700dfbd8e', 'namePublication': 'IT Technologie II', 'place': 'Prague', 'publishedDate': '2023-04-24T00:00:00', 'valid': True, 'reference': 'https://google.com', 'typePublication': 'Článek J'}
{'id': 'b5cf7f26-5652-4fb2-b930-ecc959c005f4', 'namePublication': 'IT Technologie III', 'place': 'Prague', 'publishedDate': '2023-04-24T00:00:00', 'valid': True, 'reference': 'https://google.com', 'typePublication': 'Článek J'}
<generator object flatten at 0x000001C0A4AEB7D0>


In [2]:
import json

In [3]:

with open('fakepublikace.json', 'r') as file:
    publications = json.load(file)

In [4]:
from collections import defaultdict

In [7]:
author_stats = defaultdict(lambda: {"authorTotalContribute": 0, "authorTotalPublications": 0})

for publication in publications:
    for author in publication["authors"]:
        author_name = author["user"]["name"]
        share = author["share"]
        author_stats[author_name]["authorTotalContribute"] += share
        author_stats[author_name]["authorTotalPublications"] += 1

# Calculate average contribute for each author
for author_name, stats in author_stats.items():
    author_total_contribute = stats["authorTotalContribute"]
    author_total_publications = stats["authorTotalPublications"]
    if author_total_publications > 0:
        stats["authorAverageContribute"] = author_total_contribute / author_total_publications
    else:
        stats["authorAverageContribute"] = 0

print(dict(author_stats))

{'Tai': {'authorTotalContribute': 2.1500000000000004, 'authorTotalPublications': 5, 'authorAverageContribute': 0.43000000000000005}, 'Messi': {'authorTotalContribute': 2.1399999999999997, 'authorTotalPublications': 5, 'authorAverageContribute': 0.42799999999999994}, 'Emma': {'authorTotalContribute': 3.96, 'authorTotalPublications': 9, 'authorAverageContribute': 0.44}, 'David': {'authorTotalContribute': 2.07, 'authorTotalPublications': 7, 'authorAverageContribute': 0.2957142857142857}, 'Lam': {'authorTotalContribute': 6.9799999999999995, 'authorTotalPublications': 13, 'authorAverageContribute': 0.5369230769230768}, 'Jack': {'authorTotalContribute': 2.54, 'authorTotalPublications': 8, 'authorAverageContribute': 0.3175}, 'Bob': {'authorTotalContribute': 4.81, 'authorTotalPublications': 10, 'authorAverageContribute': 0.481}, 'Hiep': {'authorTotalContribute': 5.3500000000000005, 'authorTotalPublications': 8, 'authorAverageContribute': 0.6687500000000001}}


In [8]:
author_stats=dict(author_stats)
print(author_stats)

{'Tai': {'authorTotalContribute': 2.1500000000000004, 'authorTotalPublications': 5, 'authorAverageContribute': 0.43000000000000005}, 'Messi': {'authorTotalContribute': 2.1399999999999997, 'authorTotalPublications': 5, 'authorAverageContribute': 0.42799999999999994}, 'Emma': {'authorTotalContribute': 3.96, 'authorTotalPublications': 9, 'authorAverageContribute': 0.44}, 'David': {'authorTotalContribute': 2.07, 'authorTotalPublications': 7, 'authorAverageContribute': 0.2957142857142857}, 'Lam': {'authorTotalContribute': 6.9799999999999995, 'authorTotalPublications': 13, 'authorAverageContribute': 0.5369230769230768}, 'Jack': {'authorTotalContribute': 2.54, 'authorTotalPublications': 8, 'authorAverageContribute': 0.3175}, 'Bob': {'authorTotalContribute': 4.81, 'authorTotalPublications': 10, 'authorAverageContribute': 0.481}, 'Hiep': {'authorTotalContribute': 5.3500000000000005, 'authorTotalPublications': 8, 'authorAverageContribute': 0.6687500000000001}}


In [9]:
mappersF = {
    "id": "id",
    "namePublication": "name",
    "typePublication": "publicationtype.name",
    "authorsCounts":"authorsCounts",
    "place":"place",
    "publishedDate":"publishedDate",
    "valid":"valid",
    "reference":"reference",
    "authorName":"authors.user.name",
    "authorOrder":"authors.order",
    "authorContribute":"authors.share"

}

In [10]:
async def fullPipeF():

    result = publications
    # flatData = flatten(resultMapped, {}, mappersF)
    

    resultMapped = list(map(lambda project: {**project, "authorsCounts": len(project.get("authors", []))}, result))
    # print(resultMapped)
    flatData = flatten(resultMapped, {}, mappersF)
    dataflat=list(flatData)
    for item in dataflat:
       author_name = item["authorName"]
       if author_name in author_stats:
         item.update(author_stats[author_name])


    # print(dataflat)

    seen = set()
    data_filtr_duplicated = []
    for item in dataflat:
        # Convert the dict item to a tuple of its (key, value) pairs
        item_tuple = tuple(item.items())  
        if item_tuple not in seen:
            seen.add(item_tuple)
            data_filtr_duplicated.append(item)

    filtered_data = [item for item in data_filtr_duplicated if item.get("namePublication") is not None]

    with open('fullpipefake.json', 'w', encoding='utf-8') as file:
        json.dump(filtered_data, file, ensure_ascii=False, indent=4)

    # with open('dataflatcheck.json', 'w', encoding='utf-8') as file:
    #     json.dump(dataflat, file, ensure_ascii=False, indent=4)
     
    for item in flatData:
        print(item)

    return flatData

In [13]:
resultF =await fullPipeF()


In [137]:
resultF =await fullPipeF2()

In [42]:
print(type(resultF))

<class 'generator'>


In [43]:
print(list(resultF))

[]
